In [1]:
from octis.models.LDA import LDA
from octis.models.NeuralLDA import NeuralLDA
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.preprocessing.preprocessing import Preprocessing

from spacy.lang.el.stop_words import STOP_WORDS as el_stop

In [2]:
import pandas as pd
import string

In [3]:
# speeches_df = pd.read_csv('data/data_speeches.csv')
statements_df = pd.read_csv('data/data_statements.csv')

# merge the two datasets
# df = pd.concat([speeches_df, statements_df], ignore_index=True)
df = statements_df

In [4]:
df.sample(5)

,date,id,url,title,text
19,2024-02-15,33634,https://www.primeminister.gr/2024/02/15/33634,Ενημερωτικό σημείωμα για τη συνάντηση του Πρωθ...,Ο Πρωθυπουργός Κυριάκος Μητσοτάκης συναντήθηκε...
138,2023-02-15,31311,https://www.primeminister.gr/2023/02/15/31311,Συνάντηση του Πρωθυπουργού Κυριάκου Μητσοτάκη ...,"Κυρίες και κύριοι, είναι πραγματικά δύσκολο να..."
276,2022-01-26,28429,https://www.primeminister.gr/2022/01/26/28429,Εισαγωγική τοποθέτηση του Πρωθυπουργού Κυριάκο...,Καλημέρα σας. Θέλω να ξεκινήσω ζητώντας μία πρ...
104,2023-07-13,32175,https://www.primeminister.gr/2023/07/13/32175,Δήλωση του Πρωθυπουργού Κυριάκου Μητσοτάκη κατ...,"Χαίρομαι πολύ κυρία Υπουργέ, κύριε Aναπληρωτά,..."
329,2021-10-11,27679,https://www.primeminister.gr/2021/10/11/27679,Ενημερωτικό σημείωμα για τη συνάντηση του Πρωθ...,Ο Πρωθυπουργός Κυριάκος Μητσοτάκης συναντήθηκε...


In [5]:
# drop date, id, url, title 
df = df.drop(columns=['date', 'id', 'url', 'title'])
df.sample(5)

,text
45,Η Σύνοδος Κορυφής η οποία θα ξεκινήσει σε λίγο...
75,Ο Πρωθυπουργός Κυριάκος Μητσοτάκης επισκέφθηκε...
41,"Σε λίγο αφήνουμε πίσω το 2023, των δοκιμασιών ..."
354,Ο Πρωθυπουργός Κυριάκος Μητσοτάκης συναντήθηκε...
114,«Σήμερα θα καθορίσουμε την κυβερνητική πολιτικ...


In [6]:
# export the merged dataset
df.to_csv('data/data_merged.csv', index=False)

In [7]:
# Initialize preprocessing
preprocessor = Preprocessing(vocabulary=None, language='greek', 
                             remove_stopwords_spacy=True,
                             lemmatize=True, lowercase=True, 
                             punctuation=string.punctuation, remove_punctuation=True, remove_numbers=True,
                             max_df=0.2, min_df=0.01, min_chars=4,  min_words_docs=20, 
                             verbose=True, num_processes=8, save_original_indexes=True)
# preprocess
dataset = preprocessor.preprocess_dataset(documents_path='data/data_merged.csv')

  0%|          | 0/582 [00:00<?, ?it/s]

created vocab
4208
words filtering done


In [8]:
# save 
dataset.save('data/data_merged_preprocessed')

In [55]:
model = NeuralLDA(num_topics=25, num_layers=4, num_neurons=256)

output = model.train_model(dataset)

print(*list(output.keys()), sep="\n")

Epoch: [1/100]	Samples: [305/30500]	Train Loss: 1444.5303278688525	Time: 0:00:00.045838
Epoch: [1/100]	Samples: [66/6600]	Validation Loss: 1241.4580318566525	Time: 0:00:00.003338
Epoch: [2/100]	Samples: [610/30500]	Train Loss: 1443.8265496926228	Time: 0:00:00.037000
Epoch: [2/100]	Samples: [66/6600]	Validation Loss: 1259.1705914121685	Time: 0:00:00.004044
Epoch: [3/100]	Samples: [915/30500]	Train Loss: 1440.0033299180327	Time: 0:00:00.032121
Epoch: [3/100]	Samples: [66/6600]	Validation Loss: 1265.2659431226325	Time: 0:00:00.002990
Epoch: [4/100]	Samples: [1220/30500]	Train Loss: 1442.3022797131148	Time: 0:00:00.030780
Epoch: [4/100]	Samples: [66/6600]	Validation Loss: 1253.6546630859375	Time: 0:00:00.004241
Epoch: [5/100]	Samples: [1525/30500]	Train Loss: 1447.5812243852458	Time: 0:00:00.031492
Epoch: [5/100]	Samples: [66/6600]	Validation Loss: 1247.0808549360795	Time: 0:00:00.002955
Epoch: [6/100]	Samples: [1830/30500]	Train Loss: 1432.5355532786884	Time: 0:00:00.029653
Epoch: [6/100]

In [56]:
output.keys()

dict_keys(['topics', 'topic-document-matrix', 'topic-word-matrix', 'test-topic-document-matrix'])

In [57]:
for t in output['topics']:
  print(" ".join(t))

δεκέμβριο ηνωμένο γεωργιάδης πυροσβεστικής λίνα άγκυρα αγίας πολιτείες διπλωματικό αττικής
χαρά μεγάλο κάνουν αυτόν πατρίδα θέση τιμές προφανώς καλή βάση
πλέον συνεργασίας είχαν περίπτωση ελληνικής πλευρά σημερινή εντός φυσικό κάποιες
είτε κυρίως χωρίς σημείωσε ηνωμένο αναβάθμιση δούμε σύσκεψη διεθνή δυσκολίες
γίνεται διότι πάνω στήριξη πρώτο ρόλο γρήγορα αντιμετώπιση λίγο τονίσω
δεκέμβριο κυριακή αττικής χρήστο πυροσβεστικής δήμαρχο οικονόμου ηνωμένο αριστοτελία γερμανίας
ακόμη μιας εκεί όλων άλλες ευχαριστήσω επιχειρήσεις πρώτα φορές εκτός
συνάντησης καλύτερη γίνονται κοινού θέλουμε γαλλίας συζητήσεις μέρους ναυτικού συνέπεια
κυριακή γεωργιάδης μπρατάκος διοικητής ευρωζώνη θεόδωρος αριστοτελία πειραιά αττικής δημαρχείο
μπορέσουμε σημαντικές έργο έργα συμπολίτες τελευταία λόγω πράγματι ολόκληρη αυτούς
επίσκεψη καλύτερο λαού μήνες συνεχίσουμε χρειάζεται διμερείς πορεία λόγω πρωί
πάρα αφορούν πρόοδο ευρύτερη ηγέτες έμφαση διμερών ενέργεια θέσει ελληνικές
σχέση πατρίδα έκανε ενεργειακή κ

In [63]:
npmi = Coherence(texts=dataset.get_corpus(), topk=5, measure='c_npmi')

In [64]:
topic_diversity = TopicDiversity(topk=5)

In [65]:
topic_diversity_score = topic_diversity.score(output)
print("Topic diversity: "+str(topic_diversity_score))

npmi_score = npmi.score(output)
print("Coherence: "+str(npmi_score)) 

Topic diversity: 0.92
Coherence: nan


In [66]:
import pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()

In [67]:
pyLDAvis.lda_model.prepare(model, dataset.get_corpus(), dataset.get_vocabulary())

AttributeError: 'numpy.ndarray' object has no attribute 'get_feature_names_out'